In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
url = 'https://raw.githubusercontent.com/vijaycse/Amazon_Vine_Analysis/master/amazon_reviews_public_vine_table.csv'
ratings = pd.read_csv(url, index_col=0)

In [4]:
ratings.head(10)

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R26MV8D0KG6QI6,5.0,0.0,0.0,N,Y
R1OF8GP57AQ1A0,5.0,0.0,0.0,N,Y
R3VDC1QB6MC4ZZ,5.0,0.0,0.0,N,N
R12FA3DCF8F9ER,5.0,0.0,0.0,N,Y
RTWHVNV6X4CNJ,5.0,0.0,0.0,N,Y
RIG9AWFOGRDVO,2.0,1.0,1.0,N,Y
R1S1XSG4ZCHDGS,5.0,1.0,1.0,N,Y
RB15NBVY5ELVW,5.0,2.0,2.0,N,Y
R56358YM1ZJ7I,5.0,0.0,0.0,N,N


In [5]:
# filter DataFrame for total_votes above or equal to 20
df1 = ratings[(ratings["total_votes"] >= 20)]
df1

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R1OAZUG90XPU2U,5.0,250.0,274.0,N,Y
R23VPTUJ8I8NR5,1.0,28.0,30.0,N,Y
R33VIWRD2X5IA9,1.0,0.0,20.0,N,Y
R6SRJFJ5YH4UM,5.0,23.0,26.0,N,Y
R3DOS07BIMP4DJ,2.0,32.0,42.0,N,N
...,...,...,...,...,...
R1IQNIT2UFQE97,5.0,26.0,27.0,N,N
RIQSTVHD4MQHH,5.0,21.0,23.0,N,N
RZOSXUI2V2LO7,5.0,23.0,23.0,N,N


In [6]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
df2 = df1[((df1["helpful_votes"] / df1["total_votes"]) >= 0.5)]
df2.head()

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R1OAZUG90XPU2U,5.0,250.0,274.0,N,Y
R23VPTUJ8I8NR5,1.0,28.0,30.0,N,Y
R6SRJFJ5YH4UM,5.0,23.0,26.0,N,Y
R3DOS07BIMP4DJ,2.0,32.0,42.0,N,N
R30PGWX4HN1CFG,1.0,85.0,95.0,N,Y


In [7]:
# Create paid vine DataFrame
paid_df = df2[(df2["vine"] =='Y')]
paid_df.head(5)

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R3MKO875WGIEBJ,5.0,34.0,37.0,Y,N
R3OPNTK61FQ7MP,5.0,32.0,37.0,Y,N
RMDRU8I773X5U,4.0,22.0,25.0,Y,N
RR83RGWFEFZCL,3.0,61.0,64.0,Y,N
R2TMZ7GV8SPIRV,4.0,23.0,23.0,Y,N


In [20]:
# Create unpaid vine DataFrame
unpaid_df = df2[(df2["vine"] =='N')]
unpaid_df.head(5)

,star_rating,helpful_votes,total_votes
count,28287.000000,28287.000000,28287.000000
mean,3.689327,51.027398,57.090466
std,1.673500,207.980370,216.969892
min,1.000000,10.000000,20.000000
25%,2.000000,21.000000,24.000000
50%,5.000000,28.000000,32.000000
75%,5.000000,45.000000,51.000000
max,5.000000,23755.000000,24170.000000


In [33]:
unpaid_df.describe()

,star_rating,helpful_votes,total_votes
count,28287.000000,28287.000000,28287.000000
mean,3.689327,51.027398,57.090466
std,1.673500,207.980370,216.969892
min,1.000000,10.000000,20.000000
25%,2.000000,21.000000,24.000000
50%,5.000000,28.000000,32.000000
75%,5.000000,45.000000,51.000000
max,5.000000,23755.000000,24170.000000


In [34]:
paid_df.describe()

,star_rating,helpful_votes,total_votes
count,61.000000,61.000000,61.000000
mean,3.606557,42.852459,48.016393
std,1.345098,44.679539,46.452302
min,1.000000,11.000000,20.000000
25%,3.000000,21.000000,24.000000
50%,4.000000,29.000000,33.000000
75%,5.000000,37.000000,42.000000
max,5.000000,253.000000,258.000000


In [31]:
# paid total number of reviews
#paid_df.count
total_paid_reviews = len(paid_df.value_counts())
total_paid_reviews

58

In [32]:
# unpaid total number of reviews
total_unpaid_reviews = len(unpaid_df.value_counts())
total_unpaid_reviews

8498

In [39]:
# paid 5-star reviews
paid_five_star_reviews_df = paid_df[paid_df["star_rating"] == 5.0]
paid_five_star_reviews_df

total_five_star_paid_reviews = paid_five_star_reviews_df['star_rating'].count() 
total_five_star_paid_reviews

20

In [40]:
# paid 5-star reviews percentage
paid_five_star_percent = (total_five_star_paid_reviews / total_paid_reviews) * 100
paid_five_star_percent

34.48275862068966

In [41]:
# unpaid 5-star reviews
unpaid_five_star_reviews_df = unpaid_df[unpaid_df["star_rating"] == 5.0]
unpaid_five_star_reviews_df

total_five_star_unpaid_reviews = unpaid_five_star_reviews_df['star_rating'].count() 
total_five_star_unpaid_reviews

15689

In [42]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (total_five_star_unpaid_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

184.61991056719228